# SQUAD 1.1

## Setup

In [ ]:
#all_slow

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq transformers datasets wandb 
    !pip install -Uqq fastcore sentencepiece
    !pip install -Uqq --no-deps fastai
    !pip install -q git+git://github.com/aikindergarten/fasthugs.git

In [ ]:
from fastai.text.all import *
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

In [ ]:
task_name = 'squad'
model_name = 'distilbert-base-uncased'

max_len = 512
bs = 8
val_bs = 16

## Data

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
squad_metric = load_metric(task_name)

In [ ]:
train_ds = load_dataset(task_name, split='train[:10%]')
valid_ds = load_dataset(task_name, split='validation[:10%]')

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4fffa6cf76083860f85fa83486ec3028e7e32c342c218ff2a620fc6b2868483a)
Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4fffa6cf76083860f85fa83486ec3028e7e32c342c218ff2a620fc6b2868483a)


In [ ]:
train_ds[0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [ ]:
# train_ds = train_ds.select(range(1000))
# valid_ds = valid_ds.select(range(200))
len(train_ds), len(valid_ds)

(8760, 1057)

based on huggingface example https://github.com/huggingface/transformers/blob/master/examples/question-answering/run_qa.py

TODO: check on squad_v2

In [ ]:
# Preprocessing the datasets.
# Preprocessing is slighlty different for training and evaluation.
column_names = train_ds.column_names

question_column_name = "question" #if "question" in column_names else column_names[0]
context_column_name = "context" #if "context" in column_names else column_names[1]
answer_column_name = "answers" #if "answers" in column_names else column_names[2]

# Padding side determines if we do (question|context) or (context|question).
tokenizer = AutoTokenizer.from_pretrained(model_name)
pad_on_right = tokenizer.padding_side == "right"

max_seq_length = min(max_len, tokenizer.model_max_length)

# Training preprocessing
def prepare_train_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        return_attention_mask=False,
        max_length=max_seq_length,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=False,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [ ]:
train_ds = train_ds.map(
    prepare_train_features,
    batched=True,
    num_proc=2,
    remove_columns=column_names
)

In [ ]:
len(train_ds)

8769

In [ ]:
train_ds[0].keys()

dict_keys(['end_positions', 'input_ids', 'start_positions'])

In [ ]:
x, start_, end_ = train_ds[0]['input_ids'], train_ds[0]['start_positions'], train_ds[0]['end_positions']
tokenizer.decode(x[start_:end_], skip_special_tokens=True)

'saint bernadette soubiro'

In [ ]:
from fasthugs.data import TransformersTextBlock

In [ ]:
valid_idx = L(range(8000, len(train_ds)))

In [ ]:
blocks = (
    TransformersTextBlock(model_name, preprocessed=True), 
    CategoryBlock(vocab=list(range(max_len))),
    CategoryBlock(vocab=list(range(max_len)))
)

dblock = DataBlock(blocks=blocks,
                   get_y=[ItemGetter('start_positions'), ItemGetter('end_positions')],
                   splitter=IndexSplitter(valid_idx), # TODO use valid ds
                   n_inp=1)

In [ ]:
# dblock.summary(train_ds)

In [ ]:
%%time
dls = dblock.dataloaders(train_ds, bs=bs, val_bs=val_bs)

CPU times: user 10.8 s, sys: 182 ms, total: 11 s
Wall time: 10.9 s


In [ ]:
dls.one_batch()

({'end_positions': tensor([137,  11,  25, 168,  52,  35, 147,  85], device='cuda:0'),
  'input_ids': tensor([[  101,  2000,  3183,  ...,     0,     0,     0],
          [  101,  2043,  2001,  ...,     0,     0,     0],
          [  101,  2054,  2194,  ...,     0,     0,     0],
          ...,
          [  101,  2054,  2001,  ...,     0,     0,     0],
          [  101,  2129,  2172,  ...,     0,     0,     0],
          [  101, 16575,  1010,  ...,  7967,  1012,   102]], device='cuda:0'),
  'start_positions': tensor([130,  10,  22, 167,  50,  30, 146,  82], device='cuda:0'),
  'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')},
 TensorCategory([130,  10,  22, 167,  50,  30, 146,  82], device='cuda:0'),
 TensorCategory([137,  11,  25, 168,  52,  35, 147,  85], device='cuda:0')

In [ ]:
# Validation preprocessing
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=False,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
valid_ds = valid_ds.map(prepare_validation_features,
                        batched=True,
                        num_proc=2,
                        remove_columns=column_names)

In [ ]:
# valid_ds[0]

In [ ]:
valid_ds.column_names

['attention_mask', 'example_id', 'input_ids', 'offset_mapping']

## Training Setup

## Training

In [ ]:
from fasthugs.learner import TransLearner

In [ ]:
def process_example(start_indexes, end_indexes, start_probs, end_probs, max_answer_length):
    # process single example
    candidates = []
    for start_index in start_indexes:
        for end_index in end_indexes:
            if (start_index > end_index or
                end_index - start_index + 1 > max_answer_length or
                start_index >= len(start_probs) or
                end_index >= len(start_probs)):
                continue
            # collect plausable answers in candidates
            candidates.append({'prob':start_probs[start_index] * end_probs[end_index],
                            'start_index':start_index,
                            'end_index':end_index})
    # order candidates by probability
    candidates = sorted(candidates, key= lambda x : x['prob'], reverse=True)

    #TODO add option for null answerfor squad_v2
    # simple case answer always exists
    if candidates:
        best = candidates[0]
        best_answer = (best['start_index'], best['end_index'])
    else:
        best_answer = (0, 0)
    return best_answer

In [ ]:
def qa_exact_match(preds, *targs, n_best_size=10, max_answer_length=20):
    start_targs, end_targs = targs
    start_probs, end_probs = preds[0].softmax(-1), preds[1].softmax(-1)
    start_indexes = start_probs.argsort(descending=True)[:, :n_best_size].tolist()
    end_indexes = end_probs.argsort(descending=True)[:, :n_best_size].tolist()
    bs = start_targs.size(0)
    # iterate over samples
    best_preds = [process_example(start_indexes[i], end_indexes[i], start_probs[i], end_probs[i], max_answer_length) for i in range(bs)]
    best_preds = tensor(best_preds).to(start_targs.device)
    return ((best_preds[:,0] == start_targs)*(best_preds[:,1] == end_targs)).float().mean()

In [ ]:
exact_match = AvgMetric(partial(qa_exact_match, n_best_size=10, max_answer_length=20))

In [ ]:
# learn.show_training_loop()

### Finetuning

In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
learn = TransLearner(dls,
                model,
                opt_func=RAdam,
                metrics=exact_match).to_fp16()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
learn.fit_one_cycle(2, 2e-4,)

epoch,train_loss,valid_loss,qa_exact_match,time
0,1.980754,3.172352,0.293888,03:31
1,1.066328,2.692447,0.345904,03:36


## TODO: complete evaluation as done in reference script

In [ ]:
idxs = dls.train.get_idxs()
type(idxs)

iterator

In [ ]:
tensor(list(idxs))

tensor([   0, 7413, 6918,  ..., 1029, 3761,  141])

In [ ]:
inps, preds, targs = learn.get_preds(ds_idx=0, with_input=True, with_decoded=False, reorder=True)

RuntimeError: Could not infer dtype of iterator

In [ ]:
dls.train.get_idxs()

In [ ]:
inps, preds, targs = learn.get_preds(ds_idx=0, with_input=True, with_decoded=False, reorder=False)

In [ ]:
preds[0][0]

tensor([-7.7461, -8.5859, -8.4922, -8.5234, -7.7656, -8.3594, -9.1797, -8.8750,
        -8.7891, -9.1328, -8.9062, -9.0312, -7.9375, -9.1797, -9.2188, -8.7188,
        -6.4531, -3.3184, -8.4453, -6.8867, -3.1777, -3.8633, -6.7383, -4.7109,
        -2.6465, -6.0664, -6.2812, -6.9805, -4.6992, -4.9688, -5.1055, -8.3125,
        -7.0195, -2.4941, -3.8730, -6.7930, -1.8848, -2.3262, -3.3398, -9.4688,
        -5.6523, -6.0781, -6.0977, -4.9414, -6.7188, -7.5938, -7.2617, -9.2109,
        -6.3867, -6.7852, -7.0742, -9.2891, -8.2734, -7.6641, -6.5742, -7.5625,
        -5.4844, -6.1758, -6.0430, -8.9453, -5.0977, -8.3750, -7.7578, -8.5938,
        -9.2031, -9.4062, -8.5312, -7.5547, -7.9844, -7.0312, -4.8438, -8.4297,
        -7.7148, -7.2617, -7.5781, -8.2500, -7.4844, -8.4609, -7.4805, -8.1875,
        -8.5859, -6.9570, -7.0000, -7.0625, -8.8281, -7.1250, -6.0039, -9.3516,
        -7.4062, -7.1250, -7.8477, -6.9180, -6.7539, -7.3672, -6.7266, -6.4570,
        -7.6094, -5.7344, -4.8711, -6.56

In [ ]:
learn.get_preds??

Signature:
learn.get_preds(
    ds_idx=1,
    dl=None,
    with_input=False,
    with_decoded=False,
    with_loss=False,
    act=None,
    inner=False,
    reorder=True,
    cbs=None,
    save_preds=None,
    save_targs=None,
    concat_dim=0,
)
Docstring: Get the predictions and targets on the `ds_idx`-th dbunchset or `dl`, optionally `with_input` and `with_loss`
Source:   
    @delegates(GatherPredsCallback.__init__)
    def get_preds(self, ds_idx=1, dl=None, with_input=False, with_decoded=False, with_loss=False, act=None,
                  inner=False, reorder=True, cbs=None, **kwargs):
        if dl is None: dl = self.dls[ds_idx].new(shuffled=False, drop_last=False)
        else:
            try: len(dl)
            except TypeError as e:
                raise TypeError("`dl` is something other than a single `DataLoader` object")
        if reorder and hasattr(dl, 'get_idxs'):
            idxs = dl.get_idxs()
            dl = dl.new(get_idxs = _ConstantFunc(idxs))
        cb = Gat

In [ ]:
inps = inps['input_ids']
inps[0]

tensor([  101,  2000,  3183,  2106,  1996,  6261,  2984,  9382,  3711,  1999,
         8517,  1999, 10223, 26371,  2605,  1029,   102,  6549,  2135,  1010,
         1996,  2082,  2038,  1037,  3234,  2839,  1012, 10234,  1996,  2364,
         2311,  1005,  1055,  2751,  8514,  2003,  1037,  3585,  6231,  1997,
         1996,  6261,  2984,  1012,  3202,  1999,  2392,  1997,  1996,  2364,
         2311,  1998,  5307,  2009,  1010,  2003,  1037,  6967,  6231,  1997,
         4828,  2007,  2608,  2039, 14995,  6924,  2007,  1996,  5722,  1000,
         2310,  3490,  2618,  4748,  2033, 18168,  5267,  1000,  1012,  2279,
         2000,  1996,  2364,  2311,  2003,  1996, 13546,  1997,  1996,  6730,
         2540,  1012,  3202,  2369,  1996, 13546,  2003,  1996, 24665, 23052,
         1010,  1037, 14042,  2173,  1997,  7083,  1998,  9185,  1012,  2009,
         2003,  1037, 15059,  1997,  1996, 24665, 23052,  2012, 10223, 26371,
         1010,  2605,  2073,  1996,  6261,  2984, 22353,  2135, 

In [ ]:
preds0, preds1 = preds[0].map(learn.loss_func[0].decodes),preds[1].map(learn.loss_func[1].decodes)

In [ ]:
for inp, sp, ep, st, et, _ in zip(inps, preds0, preds1, targs[0], targs[1], range(5)):
    print("----------------------------------------")
    print('label:      ', tokenizer.decode(inp[st:et], remove_special_tokens=False))
    print('prediction: ', tokenizer.decode(inp[sp:ep], remove_special_tokens=False))
    print(f"labels {st.item()}, {et.item()}; preds {sp.item()}, {ep.item()}")

----------------------------------------
label:       saint bernadette soubiro
prediction:  saint bernadette soubiro
labels 130, 137; preds 130, 137
----------------------------------------
label:       152
prediction:  152
labels 25, 26; preds 25, 26
----------------------------------------
label:       
prediction:  royal and western infirm
labels 35, 35; preds 65, 70
----------------------------------------
label:       hill
prediction:  hill
labels 62, 63; preds 62, 63
----------------------------------------
label:       
prediction:  
labels 59, 59; preds 59, 59


TODO add evaluation using post_proc and relevant metrics, see https://github.com/huggingface/transformers/blob/1c151283129cf4b9ae78296f8459c85c8bdf4cb7/examples/question-answering/utils_qa.py#L31

## Fin